### Gender mitigated word embedding using adversarial feature learning

#### In this project, we will try to mitiage the gender information in word embedding, based on [GloVe](https://nlp.stanford.edu/projects/glove/) and [Adversarial feature learning](https://arxiv.org/abs/1705.11122).

#### 1 GloVe Model
refer to [this](https://github.com/kefirski/pytorch_GloVe/blob/master/GloVe/glove.py) and [this](https://github.com/2014mchidamb/TorchGlove/blob/master/glove.py)

In [1]:
import torch as t
from nltk.tokenize import word_tokenize
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.init import xavier_normal
import torch.optim as optim

In [209]:
class GloVe(nn.Module):
    def __init__(self, co_oc,  embedding_size, x_max = 100, alpha = 0.75):
        """
        co_oc: co-occurrence ndarray
        """
        super(GloVe, self).__init__()
        
        self.embedding_size = embedding_size
        self.x_max = x_max
        self.alpha = alpha
        
        '''co_oc matrix'''
        self.co_oc = co_oc + 1.0
        self.vocabulary_size,_ = co_oc.shape

        self.in_embed = nn.Embedding(self.vocabulary_size, self.embedding_size)
        self.in_embed.weight = xavier_normal(self.in_embed.weight) #normalize
        
        self.in_bias = nn.Embedding(self.vocabulary_size, 1) #bias.shape =(vocabularySize,1)
        self.in_bias.weight = xavier_normal(self.in_bias.weight)
        
        self.out_embed = nn.Embedding(self.vocabulary_size, self.embedding_size)
        self.out_embed.weight = xavier_normal(self.out_embed.weight)
        
        self.out_bias = nn.Embedding(self.vocabulary_size, 1)
        self.out_bias.weight = xavier_normal(self.out_bias.weight)
        
    
    def forward(self, batch_input, batch_output):
        """
        return the loss
        """
        assert len(batch_input) == len(batch_output)
        
        batch_size = len(batch_input)

        co_occurences = np.array([self.co_oc[batch_input[i], batch_output[i]] for i in range(batch_size)])
        weights = np.array([self._weight(var) for var in co_occurences])
        
        co_occurences = Variable(t.from_numpy(co_occurences)).float() #variable can do backpropagation
        weights = Variable(t.from_numpy(weights)).float()
        
        batch_input = Variable(t.from_numpy(batch_input))
        batch_output = Variable(t.from_numpy(batch_output))
        
        input_embed = self.in_embed(batch_input)
        output_embed = self.out_embed(batch_output)
        input_bias = self.in_bias(batch_input)
        output_bias = self.out_bias(batch_output)
        
        loss = (t.pow(
            ((input_embed * output_embed).sum(1) + input_bias + output_bias).squeeze(1) - t.log(co_occurences), 2
        ) * weights).sum() / batch_size
        
        return loss 
    
    def _weight(self, x):
        return 1 if x > self.x_max else (x / self.x_max) ** self.alpha
    
    def embeddings(self):
        return self.in_embed.weight.data.cpu().numpy() + self.out_embed.weight.data.cpu().numpy()
    
        

In [196]:
def get_batch(vocab_size, batch_size):
    in_index  = np.random.choice(np.arange(vocab_size), size = batch_size, replace = False)
    out_index  = np.random.choice(np.arange(vocab_size), size = batch_size, replace = False)
    return in_index, out_index
        
    

In [198]:
context_size = 3
words_file = 'test.txt'
with open(words_file, 'r') as f:
    text = f.read().lower()
word_list = word_tokenize(text)
text_size = len(word_list)
vocab = np.unique(word_list)
vocabulary_size = len(vocab)
word2ind = {word:ind for ind,word in enumerate(vocab)}

def get_co_oc_matrix(vocabulary_size):   
    comat = np.zeros((vocabulary_size, vocabulary_size))
    for i in range(text_size): #main word
        left_context_ids = [word2ind[ind] for ind in word_list[max(0, i - context_size): i]]  #left context
        right_context_ids = [word2ind[ind] for ind in word_list[i+1: min(i+context_size+1, text_size)]] #right context
        ind = word2ind[word_list[i]]

        for left_ind, lind in enumerate(left_context_ids):
#             print(ind, lind, len(left_context_ids) - left_ind)
            comat[ind, lind] += 1./(len(left_context_ids) - left_ind) #symmetrically
            comat[lind, ind] += 1./(len(left_context_ids) - left_ind)
            
        for right_ind, rind in enumerate(right_context_ids):
#             print("right:", ind, rind, right_ind + 1)
            comat[ind, rind] += 1./(right_ind + 1)
            comat[rind, ind] += 1./(right_ind + 1)
            
    co_oc = np.transpose(np.nonzero(comat)) #non-zero index
    return comat, co_oc

In [210]:
def train_GloVe(co_oc_matrix, embeding_size, batch_size = 50, iterations = 1000):
    glove = GloVe(co_oc_matrix, embeding_size)
    optimizer = optim.Adagrad(glove.parameters(), 0.05)
    
    for i in range(iterations):
        in_data, out_data = get_batch(len(co_oc_matrix), batch_size)
        
        loss = glove(in_data, out_data)
        
        print("%s-epoch, mean loss = %s"%(str(i),str(loss.data[0])))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    word_embeddings = glove.embeddings()
    
    return word_embeddings

In [197]:
#test get_batch()
np.random.seed(1)
in_index, out_index = get_batch(len(comat), 50)
print(in_index, out_index)

[ 306  888  126  486 1022  226  742  872  200  133  309  576  101  705  493
  321  553  108  216  919  181  259  528 1048  283 1056  395  385  943  730
  673   90  795   49  691  336  298  419   94  156   80  650  886  301  981
  148  455   41   65   99] [ 485  201  562  241  193  935  670 1009  510  270   31  933  432 1044  548
   93  117  126  108  960  169  203  153  253  396 1089  690  598  429  414
  980  313 1075  805 1087  205   86  668  725  716  734  834 1001  695  431
 1040  362  111  423   65]


In [212]:
word2ind['the']

955

In [213]:
comat, co_oc = get_co_oc_matrix(vocabulary_size)
print(word2ind['was'])
print(comat.shape)

1053
(1115, 1115)


### !!!loss does not work normally

In [214]:
word_embeddings = train_GloVe(comat, 50, 20, 300)

0-epoch, mean loss = 0.0014853744069114327
1-epoch, mean loss = 0.0027708634734153748
2-epoch, mean loss = 0.0031356457620859146
3-epoch, mean loss = 0.0034540309570729733
4-epoch, mean loss = 0.002433686051517725
5-epoch, mean loss = 0.0019925106316804886
6-epoch, mean loss = 0.28705134987831116
7-epoch, mean loss = 0.0014464030973613262
8-epoch, mean loss = 0.0010627468582242727
9-epoch, mean loss = 0.09177442640066147
10-epoch, mean loss = 0.0019083479419350624
11-epoch, mean loss = 0.0882575660943985
12-epoch, mean loss = 0.0021228003315627575
13-epoch, mean loss = 0.08882273733615875
14-epoch, mean loss = 0.0888444110751152
15-epoch, mean loss = 0.001352024031803012
16-epoch, mean loss = 0.0018375886138528585
17-epoch, mean loss = 0.18161331117153168
18-epoch, mean loss = 0.0023743044584989548
19-epoch, mean loss = 0.0040374817326664925
20-epoch, mean loss = 0.08394147455692291
21-epoch, mean loss = 0.002311627846211195
22-epoch, mean loss = 0.014286595396697521
23-epoch, mean los

190-epoch, mean loss = 0.004451552405953407
191-epoch, mean loss = 0.002075175754725933
192-epoch, mean loss = 0.0032145585864782333
193-epoch, mean loss = 0.003255173098295927
194-epoch, mean loss = 0.08811486512422562
195-epoch, mean loss = 0.005282385740429163
196-epoch, mean loss = 0.00260902545414865
197-epoch, mean loss = 0.0028478153981268406
198-epoch, mean loss = 0.24867987632751465
199-epoch, mean loss = 0.003388919634744525
200-epoch, mean loss = 0.0025844776537269354
201-epoch, mean loss = 0.0014702752232551575
202-epoch, mean loss = 0.0027991468086838722
203-epoch, mean loss = 0.037564389407634735
204-epoch, mean loss = 0.003589692059904337
205-epoch, mean loss = 0.003770689247176051
206-epoch, mean loss = 0.12643033266067505
207-epoch, mean loss = 0.030492544174194336
208-epoch, mean loss = 0.0037364661693573
209-epoch, mean loss = 0.0026701653841882944
210-epoch, mean loss = 0.0028944690711796284
211-epoch, mean loss = 0.003071450162678957
212-epoch, mean loss = 0.003829

#### 2. Adversarial Feature Learning

refer to [this](https://github.com/github-pengge/adversarial_invariance_feature_learning) code and [paper](https://arxiv.org/pdf/1705.11122.pdf)

Replace the last term in Eq.(3) to the GloVe loss